### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [1]:
import keras 
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK, Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature 

In [2]:
## Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv",sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
##Split the data 
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
train_x = train.drop("quality",axis=1).values
train_y=train[['quality']].values.ravel()

##test dataset 
test_x = test.drop("quality",axis=1).values
test_y=test[['quality']].values.ravel()

##Splittint this traindata into train and validation
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [5]:
## ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    
    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)
    
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1,)
        ]
    )
    
    ##Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params["momentum"]
    ),loss="mean_squared_error", metrics=[keras.metrics.RootMeanSquaredError()] )
    
    ## Train the model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=[valid_x,valid_y],
                  epochs=epochs,
                  batch_size=64)
        
        ##Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)
        
        eval_rmse = eval_result[1]
        
        ##Log the params and result
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ##Log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)
        
        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [6]:
def objective(params):
    ##MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y, 
        test_x=test_x,
        test_y=test_y
    )
    
    return result 

In [12]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [13]:
mlflow.set_experiment("Wine Quality")
with mlflow.start_run():
    #Conduct the hyperparameteer search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    #Fetch details of the best run
    best_run = sorted(trials.results,key=lambda x: x["loss"])[0]
    
    #Log the best parameters,loss and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)
    
    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/15 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - loss: 37.5124 - root_mean_squared_error: 6.1247
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 12.3479 - root_mean_squared_error: 3.3743 - val_loss: 1.2011 - val_root_mean_squared_error: 1.0960

Epoch 2/3                                            

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.4534 - root_mean_squared_error: 1.2056
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0032 - root_mean_squared_error: 0.9995 - val_loss: 0.5055 - val_root_mean_squared_error: 0.7110

Epoch 3/3                                            

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4937 - root_mean_squared_error: 0.7027
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4828 - root_mean_squared_error: 0.6947 - val_loss: 0.4872 - val_root_mean_squared_error: 0.6980

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5157 - root_mean_squared_error: 0.7181
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

2025/06/16 21:32:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - loss: 33.7003 - root_mean_squared_error: 5.8052
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 31.4113 - root_mean_squared_error: 5.6043 - val_loss: 29.7119 - val_root_mean_squared_error: 5.4509

Epoch 2/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.5211 - root_mean_squared_error: 5.5246
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.8333 - root_mean_squared_error: 5.4619 - val_loss: 28.3976 - val_root_mean_squared_error: 5.3289

Epoch 3/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.2839 - root_mean_squared_error: 5.4115
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.8858 - root_mean_squared_error: 5.3744 - val_loss: 27.1199 - val_root_mean_squared_error: 5.2077

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss

2025/06/16 21:32:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 2s 150ms/step - loss: 31.3908 - root_mean_squared_error: 5.6027
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.0349 - root_mean_squared_error: 5.2917 - val_loss: 18.9935 - val_root_mean_squared_error: 4.3581

Epoch 2/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19.8781 - root_mean_squared_error: 4.4585
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.5867 - root_mean_squared_error: 4.1906 - val_loss: 11.8449 - val_root_mean_squared_error: 3.4416

Epoch 3/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.5831 - root_mean_squared_error: 3.5473
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.8308 - root_mean_squared_error: 3.2890 - val_loss: 7.3386 - val_root_mean_squared_error: 2.7090

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss:

...

Epoch 1/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - loss: 33.6567 - root_mean_squared_error: 5.8014
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 14.0004 - root_mean_squared_error: 3.6109 - val_loss: 1.3187 - val_root_mean_squared_error: 1.1483

Epoch 2/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0462 - root_mean_squared_error: 1.0228
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1124 - root_mean_squared_error: 1.0546 - val_loss: 1.0409 - val_root_mean_squared_error: 1.0202

Epoch 3/3                                                                      

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.8992 - root_mean_squared_error: 0.9483
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8843 - root_mean_squared_error: 0.9403 - val_loss: 0.8708 - val_root_mean_squared_error: 0.9332

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.799

2025/06/16 21:32:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:25<00:00,  6.49s/trial, best loss: 0.6980124711990356]

2025/06/16 21:32:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.06851377210125106), 'momentum': np.float64(0.844923746353454)}
Best eval rmse: 0.6980124711990356


In [18]:
##Register the model manually
mlflow.register_model(model_uri="runs:/abb1b53777ef41a3a8524b38d748bcd2/model",name="wine-quality")
model_uri="runs:/19c5d5e27c7e44f79bb8793176caaaef/model"


Registered model 'wine-quality' already exists. Creating a new version of this model...
2025/06/16 21:56:14 WARNING mlflow.tracking._model_registry.fluent: Run with id abb1b53777ef41a3a8524b38d748bcd2 has no artifacts at artifact path 'model', registering model based on models:/m-b0a05e424d874a31ba5c85958894506b instead
Created version '2' of model 'wine-quality'.


In [20]:
#Load the model as a generic fonction
loaded_model = mlflow.pyfunc.load_model(model_uri)

#Prediciton on the test data
result =loaded_model.predict(pd.DataFrame(test_x))
print(result)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[5.2664666]
 [5.013608 ]
 [5.3421354]
 [5.2969894]
 [5.423539 ]
 [5.227179 ]
 [5.0376177]
 [4.9565654]
 [5.6007166]
 [5.411365 ]
 [6.1410017]
 [5.084599 ]
 [5.304793 ]
 [5.178283 ]
 [5.254203 ]
 [6.305392 ]
 [4.938638 ]
 [5.3074746]
 [6.8447776]
 [5.2087398]
 [5.0222864]
 [5.1074376]
 [5.255598 ]
 [5.962022 ]
 [5.2011075]
 [5.4730043]
 [6.293222 ]
 [5.221441 ]
 [4.8746476]
 [5.9186435]
 [5.274924 ]
 [4.8458138]
 [5.6598454]
 [5.11927  ]
 [5.471487 ]
 [5.082288 ]
 [5.8541846]
 [5.4305243]
 [5.4868565]
 [5.627319 ]
 [5.459276 ]
 [5.046841 ]
 [6.2675977]
 [5.1620045]
 [5.562234 ]
 [5.7518167]
 [6.748941 ]
 [5.386093 ]
 [4.993676 ]
 [5.306135 ]
 [4.7691984]
 [5.283266 ]
 [5.39954  ]
 [5.740457 ]
 [5.004787 ]
 [5.2301216]
 [5.8255515]
 [5.259698 ]
 [5.6005707]
 [5.285111 ]
 [5.287522 ]
 [5.8141403]
 [5.1300454]
 [5.1237607]
 [6.630309 ]
 [5.188628 ]
 [6.3566628]
 [5.65109  ]
 [6.2609224]
 [5.254728 ]
 [5.902927 ]
 [4.718762 ]
 [5.6529307]
 [5.3211293